# MORDRED DESCRIPTORS. 
## COVID19 - ANTIVIRALS PREDICTION

In [ ]:
%matplotlib inline

import os
os.chdir('/Users/adriana/Desktop/TFM/dataset/mordred')
import time
import pyqsar
import pickle
import mordred
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt

# Descriptors
from pandas import read_csv
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import Descriptors
    
#Processing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold

# Metrics
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, f1_score, recall_score, precision_score, classification_report
from sklearn.utils import class_weight

# Machine learning
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier, StackingClassifier, RandomTreesEmbedding
from sklearn.neural_network import MLPClassifier

In [ ]:
def GetMordredFeatures (data):
    '''Gives a table of features (1825) calculated by Mordred 2D and 3D.'''
    from mordred import Calculator, descriptors
    
    calc = Calculator(descriptors, ignore_3D=False)
    values = []
    
    for smile in data.smiles:
        mol = Chem.MolFromSmiles(smile) # it creates a rdkit molecule object for each smile data
        if mol is None: continue # to avoid some problematic smiles and continue with the others
        desc = calc(mol)
        values.append(desc)
        features = pd.DataFrame(values)
    
    return features

def ProcessData(i):
    '''Replace not float values with NaN.'''
    try: 
        return float(i)
    except:
        return np.nan
    
def GetColumns (data):
    '''Gets those columns with a percentage of cells with NaN values >= 50%.'''
    columns = []
    for col in data:
        percent = data[col].isnull().sum()/data[col].isnull().count()
        if percent >= 0.5:
            columns.append(col)
    return columns

def ML_score (models, X_train, Y_train, X_test, Y_test, seed, classes = ['0','1']):
    '''Fit diferent models, predict and return models' scores'''
    ACC = 0
    AUROC = 0
    precision = 0 
    recall = 0
    f1score = 0
    
    model_name = type(models).__name__ # get model name
    start_time = time.time()
    
    # Train different models using cross validation 
    print('> Training time: %0.2f mins'% ((time.time()-start_time)/60))
        
    models.fit(X_train, Y_train)
    
    # Predict
    y_pred = models.predict(X_test)
    y_probs = models.predict_proba(X_test)[:, 1]
    model_report = classification_report(Y_test, y_pred, target_names=classes, output_dict=True, digits=3)
     
    # Scores
    ACC = accuracy_score(Y_test, y_pred)
    AUROC = roc_auc_score(Y_test, y_probs)
    precision = model_report['weighted avg']['precision']
    recall = model_report['weighted avg']['recall']
    f1score = model_report['weighted avg']['f1-score']
    
    return ACC, AUROC, precision, recall, f1score

### DESCRIPTORS CALCULATION
To create a predictive model, first we need to extract as many descriptors as we can from the smiles. 
In this case we had used Mordred libraries which extracts 1800 descriptors. 

In [ ]:
# Load datasets
antiv = read_csv('antivirals_SMILES.csv')
drugs = read_csv('DB_SMILES4prediction.csv')

In [ ]:
# Get MORDRED descriptors from smiles
antiv_mordred = pd.concat([antiv, GetMordredFeatures(antiv)], axis=1)
drugs_mordred = pd.concat([drugs, GetMordredFeatures(drugs)], axis=1)

In [ ]:
# Save descriptors 
antiv_mordred.to_csv('antiv_mordred.csv')
drugs_mordred.to_csv('drugs_mordred.csv')

### DATASET PREPROCESSING
Visualize if our datasets have smiles with NaN values and remove them. Transform every non-float character into NaN values.
Get every descriptor that has more than 50% of NaN values and remove them. Other NaN values into 0 values. 

In [ ]:
# Load feature datasets
antiv_mordred = read_csv('antiv_mordred.csv', low_memory=False) #to solve different column types
antiv = antiv_mordred.copy() # train

drugs_mordred = read_csv('drugs_mordred.csv', low_memory=False) #to solve different column types
drugs = drugs_mordred.copy() # predict

In [ ]:
# Split ids and features
a = antiv.loc[:,'0':]
id_a = antiv.loc[:,:'Class']

d = drugs.loc[:,'0':]
id_d = drugs.loc[:,:'Class']

# Replace different column types with NaN values
a = a.applymap(ProcessData)
d = d.applymap(ProcessData)

# Restore datasets
antiv = pd.concat([id_a, a], axis=1)
drugs = pd.concat([id_d, d], axis=1)

In [ ]:
# First aproximation: any NaN value?
print('Has Antivirals dataset NaN values?', antiv.isnull().values.any()) #true
print('>> Columns with NaN: ', antiv.isnull().any().sum(), ' / ', len(antiv.columns))
print('>> Number of data points with NaN: ', antiv.isnull().any(axis=1).sum(), ' / ', len(antiv))
print('>> Number of rows with all NaN values: ', antiv.loc[:,'0':].isnull().all(axis=1).sum())

print('\nHas Drugs dataset NaN values?', drugs.isnull().values.any()) #true
print('>> Columns with NaN: ', drugs.isnull().any().sum(), ' / ', len(drugs.columns))
print('>> Number of data points with NaN: ', drugs.isnull().any(axis=1).sum(), ' / ', len(drugs))
print('>> Number of rows with all NaN values: ', drugs.loc[:,'0':].isnull().all(axis=1).sum())

In [ ]:
# Select columns to drop from antivirals dataset
to_drop = GetColumns(antiv)

# Drop the same columns in each dataset
antiv.drop(to_drop, axis=1, inplace=True) #same columns are removed
drugs.drop(to_drop, axis=1, inplace=True) #same columns are removed

In [ ]:
# Second aproximation 
print('Has Antivirals NaN values?', antiv.isnull().values.any()) #true
print('Has Drugs NaN values?', drugs.isnull().values.any()) #true

# Replace any NaN value with 0
antiv = antiv.fillna(0)
drugs = drugs.fillna(0)

In [ ]:
# Save preprocessed files
antiv.to_csv('antiv_prepro_mordred.csv', index_label=False)
drugs.to_csv('drugs_prepro_mordred.csv', index_label=False)

### FEATURE SELECTION
In this section we're going to select the descriptors. First we're going to separate our antivirals' dataset into train (80%) and test (20%) with a random_state of 80.
Then we're going to Standardize them to get the same scale in each column. Lastly we're going to apply a feature selection method or a dimension reduction technique to delimit our dataset to maximum 50-100 descriptors.

In [ ]:
# Load datasets
input_data = read_csv('antiv_prepro_mordred.csv')
pred_data = read_csv('drugs_prepro_mordred.csv')

In [ ]:
# Remove ids
input_data = input_data.loc[:,'Class':]

In [ ]:
# Set categoricals
input_data['Class'] = pd.Categorical(input_data['Class'])

# Train and test dataset, one split 0.8 train, 0.2 test. Random_state=80
x = input_data[input_data.loc[:,'0':].columns] 
y = input_data['Class']

x_train, x_test, y_train, y_test = train_test_split(x.values, y.values, test_size=0.2, random_state=80)

print('Full dataset samples: {}'.format(input_data.shape[0]))
print('Train dataset samples: {}'.format(x_train.shape[0]))
print('Test dataset samples: {}'.format(x_test.shape[0]))

In [ ]:
# Standardize data using only train set
sc = StandardScaler().fit(x_train)
sc.get_params()

In [ ]:
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

x_train_std.mean(axis=0);
x_train_std.std(axis=0);

In [ ]:
# Transform splits from arrays into DataFrames
df_train = pd.DataFrame(x_train_std, columns=list(input_data.loc[:,'0':].columns))
df_train['Class'] = y_train

df_test = pd.DataFrame(x_test_std, columns=list(input_data.loc[:,'0':].columns))
df_test['Class'] = y_test

In [ ]:
# FEATURE SELECTION - THRESHOLD
# Remove features with low variance - th=0.0001
featFilter = VarianceThreshold(threshold=0.0001) 

X_high_variance_train = featFilter.fit_transform(x_train_std) # fit and transform train dataset

selected_features = set(list(df_train.columns[featFilter.get_support(indices=True)])) # features with high variance

print('Features with high variance: {}'.format(len(featFilter.get_support(indices=True)))) # how many with high variance

pool_features = set(list(df_train.columns)[:-1]) # all features
print('Total number of features: ', len(pool_features))

eliminated_feats = list(pool_features-selected_features)[:-1] # eliminated features
print('Eliminated features: ', len(eliminated_feats))

X_high_variance_test = featFilter.transform(x_test_std) # transform test dataset

In [ ]:
# FEATURE SELECTION - KBEST
# Selection of the K Best features - mutual information k=70
kbest = SelectKBest( , k=70)

X_kbest_train = kbest.fit_transform(X_high_variance_train, y_train)

print('Train dataset dimensions: ', X_kbest_train.shape) # 70 features

X_kbest_test = kbest.transform(X_high_variance_test)

print('Test dataset dimensions: ', X_kbest_test.shape) # 70 features

features = set(list(df_train.columns[kbest.get_support(indices=True)])) # features with high variance

In [ ]:
# Reconstruct files
df_train = pd.DataFrame(X_kbest_train, columns=features)
df_train['Class'] = y_train

df_test = pd.DataFrame(X_kbest_test, columns=features)
df_test['Class'] = y_test
df_test.head(5)

In [ ]:
# Save files
df_train.to_csv('train_mordred.csv', index_label=False)
df_test.to_csv('test_mordred.csv', index_label=False)

### MACHINE LEARNING - PREDICTIVE MODELS
#### ML with train and test subsets

In [ ]:
# Load data
train = read_csv('train_mordred.csv')
test = read_csv('test_mordred.csv')

In [ ]:
out = 'Class'
seed = 42 
np.random.seed(seed)

In [ ]:
# Train values
X_train = train.drop(out, axis=1).values
Y_train = train[out].values

# Test values
X_test = test.drop(out, axis=1).values
Y_test = test[out].values

In [ ]:
# Tested models (baseline)
models =  [LogisticRegression(random_state=seed, n_jobs=-1),
          LinearDiscriminantAnalysis(),
          QuadraticDiscriminantAnalysis(),
          
          DecisionTreeClassifier(random_state=seed),
                     
          SGDClassifier(loss='log',random_state=seed, n_jobs=-1),
          NuSVC(random_state=seed, probability=True),
          SVC(random_state=seed, probability=True),
          
          KNeighborsClassifier(n_jobs=-1),
          GaussianProcessClassifier(random_state=seed, n_jobs=-1),
          GaussianNB(),
          
          GradientBoostingClassifier(random_state=seed),
          BaggingClassifier(random_state=seed),
          AdaBoostClassifier(random_state=seed),
          RandomForestClassifier(n_jobs=-1, random_state=seed),
           
          MLPClassifier(random_state=seed),
          ]

In [ ]:
# Create a dataframe for ML scores
df_ML = pd.DataFrame(columns=['Method', 'ACC', 'AUROC', 'Precision', 'Recall', 'F1-score'])

In [ ]:
# Fit each model
for model in models:
    print("\n***", model)
    ACC, AUROC, precision, recall, f1score = ML_score(model, X_train, Y_train, X_test, Y_test, seed)
    df_ML = df_ML.append({'Method': str(type(model).__name__),
                          'ACC': float(ACC),
                          'AUROC': float(AUROC),
                          'Precision': float(precision),
                          'Recall': float(recall),
                          'F1-score': float(f1score)}, ignore_index=True)
df_ML

In [ ]:
# Save results
df_ML.to_csv('Scores_mordred.csv', index_label=False)

#### GET THE BEST MODEL
- GridSearchCV
- Manual selection

In [ ]:
gradient = GradientBoostingClassifier(random_state=seed)

In [ ]:
# GRIDSEARCH SELECTION
# GridSearchCV parameters
params = {
    'loss': ['deviance', 'exponential'],
    'n_estimators': [100, 50, 150],
    'learning_rate': [0.1, 0.001],
    'criterion': ['friedman_mse', 'mse'],
    'max_depth': [3, 5, 10],
}

In [ ]:
# GridSearch fit
gs = GridSearchCV(estimator=gradient,
                   param_grid=params, verbose=10, scoring ='roc_auc', cv=3)

gs.fit(X_train, Y_train)

In [ ]:
best_params = gs.best_params_
print(best_params)

In [ ]:
# Scores with GridSearch
ACC, AUROC, precision, recall, f1score = ML_score(gs.best_estimator_, X_train, Y_train, X_test, Y_test, seed)
print('ACC: ', ACC)
print('AUROC', AUROC)
print('Precision', precision)
print('Recall', recall)
print('F1Score', f1score)

In [ ]:
# MANUAL SELECTION
gradient_best = GradientBoostingClassifier(random_state=seed, criterion='mse', learning_rate=0.1, loss='deviance', max_depth=3, n_estimators=75)

ACC, AUROC, precision, recall, f1score = ML_score(gradient_best, X_train, Y_train, X_test, Y_test, seed)
print('ACC: ', ACC)
print('AUROC', AUROC)
print('Precision', precision)
print('Recall', recall)
print('F1Score', f1score)

In [ ]:
# Save best model
model_mordred = 'bestmodel_rdkit.sav'
pickle.dump(gradient_best, open(model_mordred, 'wb'))

### PREDICTIONS

In [ ]:
# Transform our predictions dataset with the same transformations: sc, featFilter and kbest
# Load dataset with drugs to predict
pred_data = read_csv('drugs_prepro_mordred.csv')
ids = pred_data.loc[:,'chembl_id']
pred_data = pred_data.loc[:,'Class':] 
pred_data['Class'] = pd.Categorical(pred_data['Class'])

# Remove inf values in pred_data
pred_data.info()
inf=pred_data.iloc[pred_data.values==np.inf]
pred_data = pred_data.drop(36)

# Separate values from unknown class
pred_values = pred_data[pred_data.loc[:,'0':].columns]
pred_class = pred_data['Class']

#Standardize predictions values
pred_std = sc.transform(pred_values.values)

# Remove same antivirals features
pred_highVar=featFilter.transform(pred_std) #low variance features
pred_kbest=kbest.transform(pred_highVar) #70 best features

# Reconstruct file
df_pred = pd.DataFrame(pred_kbest, columns=features)

# Save file
df_pred['chembl_id'] = ids
firstcol = df_pred.pop('chembl_id')
df_pred.insert(0, 'chembl_id', firstcol)
df_pred.to_csv('topredict.csv', index_label=False)

In [ ]:
# Read drugs dataset
data = read_csv('topredict.csv')

data_ids = data.loc[:,'chembl_id']
data_feat = data.iloc[:,1:]

In [ ]:
# Load our best model 
model = pickle.load(open('bestmodel_mordred.sav','rb')) 

In [ ]:
# Make predictions
predict = model.predict(data_feat)
prob_predict=model.predict_proba(data_feat)

In [ ]:
# Create table
dataframe = pd.DataFrame(columns=['smiles', 'ProbClass0', 'ProbClass1', 'Class'])
dataframe['smiles']=list(data['chembl_id'])
dataframe['ProbClass0']=list(prob_predict[:,0])
dataframe['ProbClass1']=list(prob_predict[:,1])
dataframe['Class']=list(predict)

In [ ]:
# Sort values by class 1
sort = dataframe.sort_values(by='ProbClass1', ascending=False)
sort.head(20)

In [ ]:
# Save top 20 predictions
sort.to_csv('predictions.csv')
top_20= sort.head(20)
top_20.to_csv('top20_mordred.csv')

### AUROC CURVE

In [ ]:
# Plot AUROC curve
fig = plt.figure()
noskill = plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill', color='lightblue')
ax = plt.gca()
metrics.plot_roc_curve(model, X_test, Y_test, ax=ax, alpha=0.8, color='blue')
plt.title('AUROC curve GradientBoostingClassifier')
ax.legend()
plt.show()

In [ ]:
# Save figure
fig.savefig('gradient_plot.png')